In [ ]:
from nuscenes.nuscenes import NuScenes
directory = r"/home/kfir/workspace/nuScenes/v1.0-trainval"
nusc = NuScenes(version="v1.0-trainval", dataroot=directory, verbose=False)

In [ ]:
%load_ext autoreload
%autoreload 2
from simulation import NuscenesSimulation

class DummyModel():
    def __init__(self):
        self.mm = True
        
scene = 690
dummy_model = DummyModel()
video_list={'video' : False, 'video_debug': False, 'video_pf': False, 'video_pf_xy': False, 'dynamic_tracker': False}
sim = NuscenesSimulation(nusc=nusc, model=dummy_model, scene_id=scene, Nmax=1200, video_list=video_list, save_processed=True)

#data = NuscenesProcessedDatabase(scene_id=scene)

In [ ]:
from nuscenes.utils.geometry_utils import transform_matrix
from pyquaternion import Quaternion
from nuscenes.utils.geometry_utils import view_points

def projectRadarOnCamera(nusc, pc, im, cs_record, min_dist: float = 1.0):

    pc.translate(-np.array(cs_record['translation']))
    pc.rotate(Quaternion(cs_record['rotation']).rotation_matrix.T)

    # Fifth step: actually take a "picture" of the point cloud.
    # Grab the depths (camera frame z axis points away from the camera).
    depths = pc.points[2, :]

    # Retrieve the color from the depth.
    coloring = depths

    # Take the actual picture (matrix multiplication with camera-matrix + renormalization).
    points = view_points(pc.points[:3, :], np.array(cs_record['camera_intrinsic']), normalize=True)

    # Remove points that are either outside or behind the camera. Leave a margin of 1 pixel for aesthetic reasons.
    # Also make sure points are at least 1m in front of the camera to avoid seeing the lidar points on the camera
    # casing for non-keyframes which are slightly out of sync.
    mask = np.ones(depths.shape[0], dtype=bool)
    mask = np.logical_and(mask, depths > min_dist)
    mask = np.logical_and(mask, points[0, :] > 1)
    mask = np.logical_and(mask, points[0, :] < im.shape[0] - 1)
    mask = np.logical_and(mask, points[1, :] > 1)
    mask = np.logical_and(mask, points[1, :] < im.shape[1] - 1)
    points = points[:, mask]
    coloring = coloring[mask]

    return points, coloring, im

nusc_map = sim.dataset.nusc_map
video_with_priors = True

t = 500
img = sim.dataset.getSyncedImage(t)
zw, covw, prior, dw, video_data, nusc_map = sim.dataset.getData(t, GT=True)
pc = sim.dataset.getEGORadarData(t, video_data["pos"], video_data["rot_gt"])

points, coloring, im = projectRadarOnCamera(nusc, pc, img, sim.dataset.cs_record_camera)
#video_data, polynoms, points, dynamic_tracks, dynamic_clusters, mm_results, translation, debug_info = data.load(t)
fig, ax = plt.subplots(1, 1, figsize=(18, 32))
ax.imshow(im)
print(points.shape)
ax.scatter(points[0, :], points[1, :], c=coloring, s=30)
ax.axis('off')